In [ ]:
# importing library
import nltk
import re 
import string
from os import listdir
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from collections import Counter # is to get the counts of all the words from 

In [1]:
# nltk.download() to download the NLTK corpous if not avilable

## reading the file

In [ ]:
# reading a negtive review text file
text_path = open("data/neg/cv000_29416.txt")
text = text_path.read()
text

## loading the file with function call

In [ ]:
def load_text(file_name):
    file = open(file_name,"r")
    text = file.read()
    return text

In [ ]:
path = "data/neg/cv000_29416.txt"
text = load_text(path)
text

## cleaning the data 

In [ ]:
def clean_text(text):
#     print("length of text file before cleaning {}".format(len(text)))
    tokens = word_tokenize(text)                                                # using word tokenizing 
    re_punct = re.compile("[%s]"%re.escape(string.punctuation))                 # for rmoving puncations
    tokens = [re_punct.sub("",w) for w in tokens]
    tokens = [w for w in tokens if w.isalpha()]                                 # takinf only alpha numeric
    stop_words = set(stopwords.words("english"))                                
    tokens = [w for w in tokens if w not in stop_words]                         # removing stopwords
    tokens = [w.lower() for w in tokens]                                    # normalizing(making all words to lower case)
    tokens = [w for w in tokens if len(w)>1]                                  # removing words length greater than 1
#     print("length of text file after cleaning {}".format(len(tokens)))
    return tokens

In [ ]:
cleaned_text = clean_text(text) #c:\cmr_python36\lib\nltk_data

## creating a bag of words(dictnory)

In [ ]:
from collections import Counter # is to get the counts of all the words from 
vocab = Counter()                     # vocabalorary

In [ ]:
def add_doc_to_vocab(filename):
    text=load_text(filename)                        # calling the load_text function
    tokens = clean_text(text)                       # calling clean_text function
    vocab.update(tokens)                            # updating the words to vocab
    return vocab

In [ ]:
# loading the directory of all the files for creating dictnory
def process_doc(directory):
    for filename in listdir(directory):
        path = directory+"/"+filename
        add_doc_to_vocab(path)

In [ ]:
path_neg = "data/neg"
path_pos = "data/pos"

In [ ]:
process_doc(path_neg)
process_doc(path_pos)

In [ ]:
vocab

In [ ]:
len(vocab)

In [ ]:
vocab.items()

In [ ]:
vocab = [k for k,c in vocab.items() if c>=2] # vocab of words greater then length 2
vocab

In [ ]:
def save_vocab(vocab,filename):
    data = "\n".join(vocab)
    file = open(filename,"w")
    file.write(data)

In [ ]:
save_vocab(vocab,"data/vocablatest.txt") # saving the vocab words to text file

In [ ]:
vocab1 = load_text("data/vocablatest.txt")
print(len(vocab1))
vocab1

# now taking the words in the data which are present in the bag of words

In [ ]:
# functin for getting the words from data which are present in bag of words 
def add_doc_to_vocab(filename):
    text=load_text(filename)
    tokens = clean_text(text)
    tokens = [w for w in tokens if w in vocab]
    return " ".join(tokens)

In [ ]:
# passing the add_doc_to_vocab for all the files
def preprocess_doc(directory,vocab): 
    lines = []
    for filename in listdir(directory): 
        path = directory+"/"+filename
        line = add_doc_to_vocab(path)
        lines.append(line)
    return lines

In [ ]:
vocab = "data/vocablatest.txt"
vocab = load_text(vocab)
vocab = set(vocab.split())
vocab

In [ ]:
# loading the cleaned data from the all the files
def load_clean_data(vocab):
    neg=preprocess_doc("data/neg",vocab)
    pos=preprocess_doc("data/pos",vocab)
    docs = neg+pos
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]  # "0" means negative review "1" means positive review
    return docs,labels

In [ ]:
docs,labes = load_clean_data(vocab)          # calling the load_clean_data

In [ ]:
len(docs),len(labes)

In [ ]:
docs[10]

In [ ]:
labes[10]

## dividing the data into train and test data

In [ ]:
# dividing the data into test and train based on the file name which starts with cv9(100 in positive 100 in negative)
def preprocess_doc(directory,vocab,is_train):
    lines = []
    for filename in listdir(directory):
        if is_train and filename.startswith("cv9"):
            continue
        if not is_train and not filename.startswith("cv9"):
            continue
        path = directory+"/"+filename
        line = add_doc_to_vocab(path)
        lines.append(line)
    return lines

In [ ]:
def load_clean_test_train_data(vocab,is_train):
    neg=preprocess_doc("data/neg",vocab,is_train)
    pos=preprocess_doc("data/pos",vocab,is_train)
    docs = neg+pos
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))] # "0" means negative review "1" means positive review
    return docs,labels

In [ ]:
train_docs,train_labels = load_clean_test_train_data(vocab,True)
test_docs,test_labels = load_clean_test_train_data(vocab,False)

In [ ]:
len(train_docs),len(train_labels)

In [ ]:
len(test_docs),len(test_labels)

## training the model

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
len(train_docs)

In [ ]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
tokenizerss = create_tokenizer(train_docs)

In [ ]:
Xtrain = tokenizerss.texts_to_matrix(train_docs,mode = "freq")
Xtest = tokenizerss.texts_to_matrix(test_docs,mode = "freq")

In [ ]:
Xtrain.shape

In [ ]:
Xtest.shape

In [ ]:
import pickle

# saving for the prediction 
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizerss, handle, protocol=pickle.HIGHEST_PROTOCOL)



## sentement analysis

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
def s_model(input_shape):
    model = Sequential()
    model.add(Dense(50,input_shape=(input_shape,),activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer="adam")
#     model.compile(loss="binary_crossentropy",optimizer="adam",metrices=["accuracy"])
    return model

In [ ]:
input_shape = Xtrain.shape[1]

In [ ]:
model = s_model(input_shape)

In [ ]:
model.summary()

In [ ]:
import numpy as np
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
print(test_labels.shape),print(train_labels.shape)

In [ ]:
model.fit(Xtrain,train_labels,epochs=50)

In [ ]:
loss=model.evaluate(Xtest,test_labels)

In [ ]:
model.save("SAMODEL.h5")

In [ ]:
from tensorflow import keras
model1 = keras.models.load_model('SAMODEL.h5')

In [ ]:
# loading
def load_tokinezer():
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
        
    return tokenizer

In [ ]:
def predict_sentiment(review,vocab,tokenizer,model):
    tokens = review.split()
    tokens = [w for w in tokens if w in vocab]
    line = " ".join(tokens)
    tokenizer = load_tokinezer()
    
    encoded = tokenizer.texts_to_matrix([line],mode = "freq")
    yhat = model.predict(encoded)
    precent_pos=yhat[0,0]
    if round(precent_pos)==0:
        return (1-precent_pos), "NAGITIVE"
    return precent_pos,"POSITIVE"

In [ ]:
(percent,sentement) = predict_sentiment(text,vocab,tokenizer,model1)
print(percent,sentement)

In [ ]:
text = "this movie is a great movie enjoyed a lot"
percent,sentement = predict_sentiment(text,vocab,tokenizer,model1)
print( "sentement of the review is -> "+str(sentement) +" and condifent score is = "+str(percent))

In [ ]:
text = "movie was a horrible and there was no thriller"
percent,sentement = predict_sentiment(text,vocab,tokenizer,model1)
print(percent),print(sentement)

In [ ]:
!streamlit run streamlit.py# for ui to run